In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'Mysql@2025',
    database ='BigMart'
)

In [3]:
cursor=conn.cursor()
print(cursor)

In [4]:
df_item = pd.read_sql("SELECT * from item_info",conn)
df_item.head()

,ID,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP
0,1,FDA15,9.30,Low Fat,0.016047,Dairy,249.8090
1,2,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692
2,3,FDN15,17.50,Low Fat,0.016760,Meat,141.6180
3,4,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950
4,5,NCD19,8.93,Low Fat,0.000000,Household,53.8614


In [5]:
df_outlet = pd.read_sql("SELECT * from outlet_info",conn)
df_outlet.head()

,ID,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,2,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,3,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,4,OUT010,1998,Medium,Tier 3,Grocery Store
4,5,OUT013,1987,High,Tier 3,Supermarket Type1


In [6]:
df_sales = pd.read_sql("SELECT * from sales_info",conn)
df_sales.head()

,ID,Item_Outlet_Sales
0,1,3735.140
1,2,443.423
2,3,2097.270
3,4,732.380
4,5,994.705


In [7]:
# Merge all the df
df = df_item.merge (df_outlet, on ='ID').merge(df_sales, on ='ID')
df = df.drop('ID', axis = 1)

In [8]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8090,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.140
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.423
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.270
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.380
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.705


In [9]:
# check duplicate value
df.duplicated().sum()

np.int64(0)

In [10]:
print(f"Shape:", {df.shape})
print(f"rows: ",{df.shape[0]})
print(f"columns:", {df.shape[1]})

Shape: {(8523, 12)}
rows:  {8523}
columns: {12}


In [11]:
df.isnull().sum().any()

np.False_

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   object 
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), object(8)
memory usage: 799.2+ KB


In [13]:
df['Outlet_Establishment_Year'] = pd.to_numeric(df['Outlet_Establishment_Year'], errors='coerce')
df['Outlet_Age'] = 2025 - df['Outlet_Establishment_Year']

In [14]:
df = df.drop('Outlet_Establishment_Year', axis = 1)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8090,OUT049,Medium,Tier 1,Supermarket Type1,3735.140,26
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.423,16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.270,26
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,Medium,Tier 3,Grocery Store,732.380,27
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.705,38


In [15]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'low fat': 'Low Fat','Low_fat': 'Low Fat', 'LF': 'Low Fat', 'reg':'Regular'})
df['Item_Fat_Content'].value_counts(normalize=True)

Item_Fat_Content
Low Fat    0.647307
Regular    0.352693
Name: proportion, dtype: float64

In [16]:
# Getting the range of 'Item_Visibility'
print('Maximum Value :', df['Item_Visibility'].max())
print('Minimum Value :', df['Item_Visibility'].min())


Maximum Value : 0.328391
Minimum Value : 0.0


In [17]:
# Capping the values
df['Item_Visibility'] = np.where(df['Item_Visibility'] > 0.3, 0.3, df['Item_Visibility'])
print('Maximum Value :', df['Item_Visibility'].max())
print('Minimum Value :', df['Item_Visibility'].min())

Maximum Value : 0.3
Minimum Value : 0.0


In [18]:
# Seperate feature from target
X = df.drop('Item_Outlet_Sales',axis = 1)
y = df['Item_Outlet_Sales']

In [19]:
categorical_cols = X.select_dtypes(include ='object').columns.tolist()

In [20]:
# Preprocessing Pipeline
preprocessor = ColumnTransformer(
   transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
],

    remainder = 'passthrough'
)


In [21]:
# Defining the models
from sklearn.ensemble import RandomForestRegressor
models = {
    'GradientBoosting' : GradientBoostingRegressor(n_estimators = 200, learning_rate = 0.1, random_state = 42),
    'RandomForest' : RandomForestRegressor(n_estimators = 200, random_state = 42),
    'LinearRegression' : LinearRegression()
}


In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2, random_state =42)

In [23]:
# Evaluate the model
best_model_name = None
best_score = -np.inf
best_pipeline = None

for name, reg in models.items():
    pipeline = Pipeline(steps = [
        ('preprocessor', preprocessor),
        ('regressor', reg)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    r2 = np.round(metrics.r2_score(y_test, y_pred), 3)
    print(f"\n{name} Results :")
    print(f"R2 Score : {r2 : .3f}")

    if r2 > best_score:
        best_score = r2
        best_model_name = name
        best_pipeline = pipeline



GradientBoosting Results :
R2 Score :  0.593

RandomForest Results :
R2 Score :  0.563

LinearRegression Results :
R2 Score :  0.408


In [26]:
#saving the pickle file format of the best model
import sklearn
with open('bigmart_best_model.pkl','wb') as f:
    pickle.dump((best_pipeline,sklearn.__version__),f)